In [1]:
import torch
import torch.nn as nn
import numpy as np

class DataEmbedding(nn.Module):
    def __init__(self, feats_in: int, feats_model: int, embedding_type: str = 'fixed', frequency: str = 'h', dropout: float = 0.1):
        super(DataEmbedding, self).__init__()
        self.value_embedding = nn.Linear(feats_in, feats_model)  # [1] -> [d_model]
        self.position_embedding = PositionalEmbedding(feats_model=feats_model)
        self.temporal_embedding = TemporalEmbedding(feats_model=feats_model, embedding_type=embedding_type, frequency=frequency)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x: torch.Tensor, x_mark: torch.Tensor) -> torch.Tensor:
        # x shape: [B, T, 1]
        x_emb = self.value_embedding(x)  # [B, T, 1] -> [B, T, d_model]
        x_emb = x_emb + self.position_embedding(x)  # [B, T, d_model]

        if x_mark is not None:
            x_emb += self.temporal_embedding(x_mark)

        return self.dropout(x_emb)

class TemporalEmbedding(nn.Module):
    def __init__(self, feats_model: int, embedding_type: str = 'timeF', frequency: str = 'h'):
        super(TemporalEmbedding, self).__init__()

        # 시간 주기별 임베딩 차원
        Embed = nn.Embedding

        # 시간 주기 구성: 시간 단위에 따라 다르게 처리
        if frequency == 'h':
            self.minute_embed = None
            self.hour_embed = Embed(24, feats_model)       # 0~23
            self.weekday_embed = Embed(7, feats_model)     # 0~6
            self.day_embed = Embed(32, feats_model)        # 1~31
            self.month_embed = Embed(13, feats_model)      # 1~12
        else:
            raise NotImplementedError(f"Unsupported frequency: {frequency}")

    def forward(self, x_mark: torch.Tensor) -> torch.Tensor:
        """
        Parameters
        ----------
        x_mark : Tensor of shape [B, T, 4] → columns: hour, weekday, day, month

        Returns
        -------
        Tensor of shape [B, T, d_model]
        """
        hour_x = self.hour_embed(x_mark[:, :, 0].long())
        weekday_x = self.weekday_embed(x_mark[:, :, 1].long())
        day_x = self.day_embed(x_mark[:, :, 2].long())
        month_x = self.month_embed(x_mark[:, :, 3].long())

        return hour_x + weekday_x + day_x + month_x


class PositionalEmbedding(nn.Module):
    def __init__(self, feats_model: int, max_len: int = 5000):
        super(PositionalEmbedding, self).__init__()

        # 위치 인덱스 (0, 1, 2, ..., max_len - 1)
        pe = torch.zeros(max_len, feats_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, feats_model, 2).float() * (-np.log(10000.0) / feats_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)  # [1, max_len, d_model]
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Parameters
        ----------
        x : Tensor of shape [batch_size, seq_len, feats_model]
        """
        return self.pe[:, :x.size(1), :]


class Inception_Block_V1(nn.Module):
    def __init__(self, in_channels, out_channels, num_kernels=6, init_weight=True):
        super(Inception_Block_V1, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.num_kernels = num_kernels
        kernels = []
        for i in range(self.num_kernels):
            kernels.append(nn.Conv2d(in_channels, out_channels, kernel_size=2 * i + 1, padding=i))
        self.kernels = nn.ModuleList(kernels)
        if init_weight:
            self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        res_list = []
        for i in range(self.num_kernels):
            res_list.append(self.kernels[i](x))
        res = torch.stack(res_list, dim=-1).mean(-1)
        return res


class Inception_Block_V2(nn.Module):
    def __init__(self, in_channels, out_channels, num_kernels=6, init_weight=True):
        super(Inception_Block_V2, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.num_kernels = num_kernels
        kernels = []
        for i in range(self.num_kernels // 2):
            kernels.append(nn.Conv2d(in_channels, out_channels, kernel_size=[1, 2 * i + 3], padding=[0, i + 1]))
            kernels.append(nn.Conv2d(in_channels, out_channels, kernel_size=[2 * i + 3, 1], padding=[i + 1, 0]))
        kernels.append(nn.Conv2d(in_channels, out_channels, kernel_size=1))
        self.kernels = nn.ModuleList(kernels)
        if init_weight:
            self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        res_list = []
        for i in range(self.num_kernels + 1):
            res_list.append(self.kernels[i](x))
        res = torch.stack(res_list, dim=-1).mean(-1)
        return res



In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.fft

def FFT_for_Period(x, k=2):
    # [B, T, C]
    xf = torch.fft.rfft(x, dim=1)
    # find period by amplitudes
    frequency_list = abs(xf).mean(0).mean(-1)
    frequency_list[0] = 0
    _, top_list = torch.topk(frequency_list, k)
    top_list = top_list.detach().cpu().numpy()
    period = x.shape[1] // top_list
    return period, abs(xf).mean(-1)[:, top_list]


class TimesBlock(nn.Module):
    def __init__(self, configs):
        super(TimesBlock, self).__init__()
        self.seq_len = configs.seq_len
        self.pred_len = configs.pred_len
        self.k = configs.top_k
        # parameter-efficient design
        self.conv = nn.Sequential(
            Inception_Block_V1(configs.d_model, configs.d_ff,
                               num_kernels=configs.num_kernels),
            nn.GELU(),
            Inception_Block_V1(configs.d_ff, configs.d_model,
                               num_kernels=configs.num_kernels)
        )

    def forward(self, x):
        B, T, N = x.size()
        period_list, period_weight = FFT_for_Period(x, self.k)

        res = []
        for i in range(self.k):
            period = period_list[i]
            # padding
            if (self.seq_len + self.pred_len) % period != 0:
                length = (
                                 ((self.seq_len + self.pred_len) // period) + 1) * period
                padding = torch.zeros([x.shape[0], (length - (self.seq_len + self.pred_len)), x.shape[2]]).to(x.device)
                out = torch.cat([x, padding], dim=1)
            else:
                length = (self.seq_len + self.pred_len)
                out = x
            # reshape
            out = out.reshape(B, length // period, period,
                              N).permute(0, 3, 1, 2).contiguous()
            # 2D conv: from 1d Variation to 2d Variation
            out = self.conv(out)
            # reshape back
            out = out.permute(0, 2, 3, 1).reshape(B, -1, N)
            res.append(out[:, :(self.seq_len + self.pred_len), :])
        res = torch.stack(res, dim=-1)
        # adaptive aggregation
        period_weight = F.softmax(period_weight, dim=1)
        period_weight = period_weight.unsqueeze(
            1).unsqueeze(1).repeat(1, T, N, 1)
        res = torch.sum(res * period_weight, -1)
        # residual connection
        res = res + x
        return res

class Model(nn.Module):
    def __init__(self, configs):
        super(Model, self).__init__()
        self.configs = configs
        self.task_name = configs.task_name
        self.seq_len = configs.seq_len
        self.label_len = configs.label_len
        self.pred_len = configs.pred_len
        self.model = nn.ModuleList([TimesBlock(configs)
                                  for _ in range(configs.e_layers)])
        self.enc_embedding = DataEmbedding(configs.enc_in, configs.d_model, configs.embed, configs.freq,
                                         configs.dropout)
        self.layer = configs.e_layers
        self.layer_norm = nn.LayerNorm(configs.d_model)

        # Always initialize projection layer for regression
        self.projection = nn.Linear(configs.d_model, configs.c_out, bias=True)

    def forecast(self, x_enc, x_mark_enc, x_dec, x_mark_dec):
        # Normalization
        means = x_enc.mean(1, keepdim=True).detach()
        x_enc = x_enc - means
        stdev = torch.sqrt(torch.var(x_enc, dim=1, keepdim=True, unbiased=False) + 1e-5)
        x_enc /= stdev

        # Embedding
        enc_out = self.enc_embedding(x_enc, x_mark_enc)  # [B,T,C]

        # TimesNet blocks
        for i in range(self.layer):
            enc_out = self.layer_norm(self.model[i](enc_out))

        # Projection
        dec_out = self.projection(enc_out)

        # De-normalization
        dec_out = dec_out * stdev[:, 0, :].unsqueeze(1).repeat(1, self.seq_len, 1)
        dec_out = dec_out + means[:, 0, :].unsqueeze(1).repeat(1, self.seq_len, 1)

        return dec_out

    def forward(self, x_enc, x_mark_enc, x_dec, x_mark_dec, mask=None):
        dec_out = self.forecast(x_enc, x_mark_enc, x_dec, x_mark_dec)
        return dec_out.mean(dim=1)  # Average over time dimension to get [B, 2]


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# TimesNet 설정 클래스

def get_config(seq_len, pred_len, input_dim, output_dim):
    class Config:
        def __init__(self):
            self.task_name = 'regression'
            self.enc_in = input_dim  # Should be 1 for your case
            self.c_out = output_dim  # Should be 2 (SBP and DBP)
            self.d_model = 64
            self.d_ff = 128
            self.num_kernels = 6
            self.top_k = 3
            self.embed = 'timeF'
            self.freq = 'h'
            self.dropout = 0.1
            self.seq_len = seq_len
            self.label_len = 0
            self.pred_len = pred_len
            self.e_layers = 2

    return Config()


# 1. Load Data
file_path = "/content/drive/MyDrive/Colab Notebooks/combined_dataset.xlsx"
df = pd.read_excel(file_path)

# 2. Feature & Target Selection
X_raw = df[[str(i) for i in range(1, 2101)]].dropna()
y_sbp = df.loc[X_raw.index, 'Systolic Blood Pressure(mmHg)']
y_dbp = df.loc[X_raw.index, 'Diastolic Blood Pressure(mmHg)']

# 3. Sliding Window Transform
def sliding_window_transform(X_raw, y_sbp, y_dbp, window_size, stride):
    X_win, y_win = [], []
    for i in range(len(X_raw)):
        row = X_raw.iloc[i].values
        for start in range(0, 2100 - window_size + 1, stride):
            end = start + window_size
            X_win.append(row[start:end])
            y_win.append([y_sbp.iloc[i], y_dbp.iloc[i]])
    return np.array(X_win), np.array(y_win)

# Set window & stride
window_size = 200
stride = 200
X_win, y_win = sliding_window_transform(X_raw, y_sbp, y_dbp, window_size, stride)

# 4. Normalize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_win.reshape(-1, window_size)).reshape(X_win.shape)

# 5. Tensor Conversion (B, T, 1)
# Convert the input data to float type before passing it to the model
X_tensor = torch.tensor(X_scaled, dtype=torch.float32).unsqueeze(-1)  # (batch, seq_len, 1)
y_tensor = torch.tensor(y_win, dtype=torch.float32)
             # (batch, 2)
# 6. Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X_tensor, y_tensor, test_size=0.2, random_state=42
)

# 7. DataLoader 구성
batch_size = 64
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# Model 설정
config = get_config(seq_len=window_size, pred_len=0, input_dim=1, output_dim=2)

model = Model(config)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.L1Loss()

# Train
from tqdm import tqdm  # 진행률 표시를 위한 라이브러리

best_val_loss = float('inf')
for epoch in range(20):
    # Training Phase
    model.train()
    train_loss = 0
    train_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/20 [TRAIN]', leave=False)

    for xb, yb in train_bar:
        pred = model(xb, None, None, None)
        loss = loss_fn(pred, yb)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_bar.set_postfix({'loss': f'{loss.item():.4f}'})

    avg_train_loss = train_loss / len(train_loader)

    # Validation Phase
    model.eval()
    val_loss = 0
    val_sbp_mae = 0
    val_dbp_mae = 0

    with torch.no_grad():
        val_bar = tqdm(test_loader, desc=f'Epoch {epoch+1}/20 [VAL]', leave=False)
        for xb, yb in val_bar:
            pred = model(xb, None, None, None)
            loss = loss_fn(pred, yb)
            val_loss += loss.item()

            # Calculate MAE for SBP and DBP separately
            sbp_mae = mean_absolute_error(yb[:, 0].cpu().numpy(), pred[:, 0].cpu().numpy())
            dbp_mae = mean_absolute_error(yb[:, 1].cpu().numpy(), pred[:, 1].cpu().numpy())
            val_sbp_mae += sbp_mae
            val_dbp_mae += dbp_mae

    avg_val_loss = val_loss / len(test_loader)
    avg_sbp_mae = val_sbp_mae / len(test_loader)
    avg_dbp_mae = val_dbp_mae / len(test_loader)

    # Print epoch summary
    print(f'\nEpoch {epoch+1}/20:')
    print(f'Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}')
    print(f'Val MAE - SBP: {avg_sbp_mae:.2f} mmHg | DBP: {avg_dbp_mae:.2f} mmHg')

    # Save best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), 'best_model.pth')
        print(f'Best model saved (Val Loss: {best_val_loss:.4f})')

print('\nTraining completed!')

# Eval
model.eval()
with torch.no_grad():
    pred_test = model(X_test, None, None, None).mean(dim=1)
    mae_sbp = mean_absolute_error(y_test[:, 0].numpy(), pred_test[:, 0].numpy())
    mae_dbp = mean_absolute_error(y_test[:, 1].numpy(), pred_test[:, 1].numpy())

print(f"MAE (SBP): {mae_sbp:.2f} mmHg")
print(f"MAE (DBP): {mae_dbp:.2f} mmHg")

Epoch 1/20 [TRAIN]:  25%|██▌       | 21/83 [11:49<35:43, 34.57s/it, loss=100.9807]